## Test 1.1: take 10 images with random PWM values

In [4]:
import csv
import numpy as np
import random
import time
from PIL import Image
from imageio import imwrite,imread
import os
from pylablib.devices import Thorlabs
from PIL import Image
import PIL.ImageOps 
import matplotlib.pyplot as plt 
  
from pylablib.devices import Thorlabs
Thorlabs.list_cameras_tlcam()
cam = Thorlabs.ThorlabsTLCamera(serial="14308")
cam.set_exposure(0.0001) # set exposure to 100ms
cam.set_roi(0,1000,0,1000) # set ROI in the upper left corner


import laserbeamsize as lbs
A = np.random.randint(low=0,high=2000,size=(5,1), dtype=np.int32)   # range form 0 to 3000
with open('PWM_test.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(A)

raw_image_array = cam.grab(10) # grab 10 frames
image_array=np.array(raw_image_array)
squeezed_image_array=image_array.squeeze()
print(image_array.shape)
print(squeezed_image_array.shape)

(10, 1000, 1000)
(10, 1000, 1000)


In [8]:
print(len(squeezed_image_array))

10


In [20]:
cam.close()

In [ ]:
image_array.shape

In [ ]:
squeezed_image_array.shape

## Test 1.2: use quality analysis methods to calculate the average score

In [13]:
from scipy import stats
from scipy import optimize
from sklearn.metrics import r2_score
from PIL import Image
import glob

def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) / 4 / stddev)**2)
def get_quality_number1(squeezed_image_array):
    score=[]
    for i in range (len(squeezed_image_array)):
        x=np.linspace(0,1000,1000)
        [m,n]=np.where(squeezed_image_array[i]==np.max(squeezed_image_array[i]))
        mode_info_m = stats.mode(m, keepdims=True)  ##col
        mode_info_n = stats.mode(n, keepdims=True) ##row
        a=int(mode_info_m[0])
        b=int(mode_info_n[0])
        c=5 #box half-thickness
        y_x=0

        for j in range (a-c,a+c):
            y_x=y_x+squeezed_image_array[i][j,:]/100 #tricky here since if this number is >255 it becomes 0.
        y_x=(y_x)/c*100

        y_y=0

        for j in range (b-c,b+c):
            y_y=y_y+squeezed_image_array[i][:,j]/100
        y_y=((y_y)/c)*100

        popt_x, pcov_x = optimize.curve_fit(gaussian, x, y_x+1,maxfev=500000) # tricky here!!!!! y_x+1
        fitted_y_x=popt_x[0] * np.exp(-((x - popt_x[1]) / 4 / popt_x[2])**2)
        a_x=r2_score(y_x,fitted_y_x)
        popt_y, pcov_y = optimize.curve_fit(gaussian, x, y_y+1,maxfev=500000) # same here!!!! y_y+1
        fitted_y_y=popt_y[0] * np.exp(-((x - popt_y[1]) / 4 / popt_y[2])**2)
        a_y=r2_score(y_y,fitted_y_y)

        s=abs(a_x*a_y)
        score.append(s)
        average_score=np.mean(score)
    return average_score

s=get_quality_number1(squeezed_image_array)
print(s)

0.9655917982764729


In [17]:
print(image_array.shape)
print(squeezed_image_array.shape)

(10, 1000, 1000)
(10, 1000, 1000)


In [18]:
import laserbeamsize as lbs

def get_quality_number2(squeezed_image_array):
    score=[]
    for i in range (len(squeezed_image_array)):
        x, y, dx, dy, phi = lbs.beam_size(squeezed_image_array[i])
        Ellipticity = min(dx, dy) / max(dx, dy)
        threshold_up_area = (squeezed_image_array[i] >= np.max(squeezed_image_array[i]) / 10) + 0  # define a threshold area
        total_pixel = squeezed_image_array[i].shape[0] * squeezed_image_array[i].shape[1]
        dispersion = total_pixel / np.sum(threshold_up_area)
        quality_number = dispersion * Ellipticity
        score.append(quality_number)
        average_score=np.mean(score)
    return average_score
s=get_quality_number2(squeezed_image_array)
print(s)


6.961334535781131


In [25]:
cam.close()

## Test 1.3: test for BO loop

In [26]:
from skopt import gp_minimize
import pygad
import laserbeamsize as lbs
import numpy as np

# Use the optimal_pwm_values for controlling the deformable mirror
from pylablib.devices import Thorlabs
Thorlabs.list_cameras_tlcam()
cam = Thorlabs.ThorlabsTLCamera(serial="14308")
cam.set_exposure(0.0001) # set exposure to 100ms
cam.set_roi(0,1000,0,1000) # set ROI in the upper left corner

def objective(solution):
    A = np.array([[solution[0]], [solution[1]], [solution[2]], [solution[3]], [solution[4]]])
    with open('PWM_test.csv', 'w', newline='') as file:
        mywriter = csv.writer(file, delimiter=',')
        mywriter.writerows(A)
    time.sleep(0.1)
    raw_image_array = cam.grab(10) # grab 10 frames
    image_array=np.array(raw_image_array)
    squeezed_image_array=image_array.squeeze()
    quality_number = get_quality_number1(squeezed_image_array)
    score = -quality_number
    print(f"Score: {score}")
    return score

bounds = [(0, 4095)] * 5  # Example bounds for 5 PWM values

# Modify the following parameters as per your requirements
n_calls = 25  # Number of iterations
n_points = 10  # Number of points used for training the surrogate model

res = gp_minimize(objective, bounds, n_calls=n_calls, n_points=n_points)

optimal_pwm_values = res.x
print(optimal_pwm_values)
print(res)

Score: -0.9381345819494188
Score: -0.45192856486216765
Score: -0.4691267670401816
Score: -0.8902867781166715
Score: -0.3703091039519758
Score: -0.3807951936770517
Score: -0.6787614204760832
Score: -0.64886384878882
Score: -0.5910864112445576
Score: -0.7735733511662247
Score: -0.6762251964450953


C:\Users\plasma\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:486: RuntimeWarning: overflow encountered in matmul
  cov_x = invR @ invR.T
C:\Users\plasma\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:486: RuntimeWarning: overflow encountered in matmul
  cov_x = invR @ invR.T
C:\Users\plasma\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:486: RuntimeWarning: overflow encountered in matmul
  cov_x = invR @ invR.T


Score: -0.7184661522381075
Score: -0.37173909705330754
Score: -0.7548443597092004
Score: -0.4919275128585678


C:\Users\plasma\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:486: RuntimeWarning: overflow encountered in matmul
  cov_x = invR @ invR.T


Score: -0.5089535927412798
Score: -0.5979006073332777
Score: -0.8600332715525447
Score: -0.8509270628387992
Score: -0.7278886402384797
Score: -0.7379638216761567
Score: -0.913339356020207
Score: -0.9339243309037218
Score: -0.9313493013261285
Score: -0.9280152869927377
[3880, 2732, 2594, 1173, 518]
          fun: -0.9381345819494188
            x: [3880, 2732, 2594, 1173, 518]
    func_vals: [-9.381e-01 -4.519e-01 ... -9.313e-01 -9.280e-01]
      x_iters: [[3880, 2732, 2594, 1173, 518], [3572, 3013, 1709, 2939, 1747], [1084, 3237, 3293, 2185, 2891], [338, 2230, 3747, 3927, 2002], [2951, 2692, 3014, 383, 1520], [509, 1790, 3024, 2050, 1715], [979, 488, 717, 378, 1440], [2902, 1292, 1209, 1593, 2194], [1063, 1900, 1, 2711, 172], [1783, 298, 3444, 2550, 2524], [1237, 1701, 2568, 1969, 31], [2614, 1082, 2608, 928, 777], [1500, 3371, 3727, 3542, 2063], [2375, 2058, 1272, 3751, 1818], [4048, 2759, 1509, 1118, 450], [2339, 3231, 1296, 3417, 1345], [947, 227, 2020, 1969, 3393], [2037, 4019, 323

In [27]:
cam.close()